In [32]:
import numpy as np
import pandas as pd
from collections import defaultdict

from surprise import Reader
from surprise import Dataset
from surprise import KNNBasic
from surprise.model_selection import cross_validate

import pprint

In [33]:
ratings_df = pd.read_csv("rating.csv")
ratings_df.shape

(7813737, 3)

In [34]:
ratings_df.head(5)

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [35]:
# ratings_df["rating"].unique()

total_ratings_count = ratings_df["rating"].value_counts()
print(total_ratings_count)

rating
 8     1646019
-1     1476496
 7     1375287
 9     1254096
 10     955715
 6      637775
 5      282806
 4      104291
 3       41453
 2       23150
 1       16649
Name: count, dtype: int64


In [36]:
anime_df = pd.read_csv("anime.csv")
anime_df.shape

(12294, 7)

In [37]:
anime_df.head(5)

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [38]:
df = ratings_df.merge(anime_df.drop_duplicates(), left_on="anime_id", right_on= "anime_id", how="left")
df

,user_id,anime_id,rating_x,name,genre,type,episodes,rating_y,members
0,1,20,-1,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297.0
1,1,24,-1,School Rumble,"Comedy, Romance, School, Shounen",TV,26,8.06,178553.0
2,1,79,-1,Shuffle!,"Comedy, Drama, Ecchi, Fantasy, Harem, Magic, R...",TV,24,7.31,158772.0
3,1,226,-1,Elfen Lied,"Action, Drama, Horror, Psychological, Romance,...",TV,13,7.85,623511.0
4,1,241,-1,Girls Bravo: First Season,"Comedy, Ecchi, Fantasy, Harem, Romance, School",TV,11,6.69,84395.0
...,...,...,...,...,...,...,...,...,...
7813732,73515,16512,7,Devil Survivor 2 The Animation,"Action, Demons, Supernatural",TV,13,7.06,101266.0
7813733,73515,17187,9,Ghost in the Shell: Arise - Border:1 Ghost Pain,"Mecha, Police, Psychological, Sci-Fi",Movie,1,7.64,31747.0
7813734,73515,22145,10,Kuroshitsuji: Book of Circus,"Comedy, Demons, Fantasy, Historical, Shounen, ...",TV,10,8.37,122895.0
7813735,73516,790,9,Ergo Proxy,"Mystery, Psychological, Sci-Fi",TV,23,8.03,265005.0


In [39]:
df.shape

(7813737, 9)

In [40]:
df =df.dropna()
df

,user_id,anime_id,rating_x,name,genre,type,episodes,rating_y,members
0,1,20,-1,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297.0
1,1,24,-1,School Rumble,"Comedy, Romance, School, Shounen",TV,26,8.06,178553.0
2,1,79,-1,Shuffle!,"Comedy, Drama, Ecchi, Fantasy, Harem, Magic, R...",TV,24,7.31,158772.0
3,1,226,-1,Elfen Lied,"Action, Drama, Horror, Psychological, Romance,...",TV,13,7.85,623511.0
4,1,241,-1,Girls Bravo: First Season,"Comedy, Ecchi, Fantasy, Harem, Romance, School",TV,11,6.69,84395.0
...,...,...,...,...,...,...,...,...,...
7813732,73515,16512,7,Devil Survivor 2 The Animation,"Action, Demons, Supernatural",TV,13,7.06,101266.0
7813733,73515,17187,9,Ghost in the Shell: Arise - Border:1 Ghost Pain,"Mecha, Police, Psychological, Sci-Fi",Movie,1,7.64,31747.0
7813734,73515,22145,10,Kuroshitsuji: Book of Circus,"Comedy, Demons, Fantasy, Historical, Shounen, ...",TV,10,8.37,122895.0
7813735,73516,790,9,Ergo Proxy,"Mystery, Psychological, Sci-Fi",TV,23,8.03,265005.0


In [41]:
df.shape

(7813611, 9)

In [42]:
df = df[df.rating_x!=-1]

In [43]:
total_ratings_count = df["rating_x"].value_counts()
print(total_ratings_count)

rating_x
8     1646018
7     1375277
9     1254090
10     955712
6      637762
5      282791
4      104275
3       41437
2       23140
1       16644
Name: count, dtype: int64


In [44]:
df["rating_x"].describe()

count    6.337146e+06
mean     7.808543e+00
std      1.572440e+00
min      1.000000e+00
25%      7.000000e+00
50%      8.000000e+00
75%      9.000000e+00
max      1.000000e+01
Name: rating_x, dtype: float64

## Samplen ottaminen

In [45]:
# df = df.sample(frac=0.0001, random_state=42) 
df = df.sample(n=100000, random_state=42).reset_index(drop=True)  # Sample 6000 rows
# df = df.sample(n=100000, random_state=42).reset_index(drop=True)  # Sample 6000 rows
# df = df.head(100000)
# # len(df)

In [46]:
print(df.head())  # Check the sampled data
print(df.index)   # Ensure the index is reset (should start from 0)

   user_id  anime_id  rating_x                           name  \
0    27383      1557         6               Shounen Onmyouji   
1     5899     12365         6            Bakuman. 3rd Season   
2    31017       157         7           Mahou Sensei Negima!   
3    53005       199         8  Sen to Chihiro no Kamikakushi   
4    25989     10721         9             Mawaru Penguindrum   

                                               genre   type episodes  \
0  Action, Demons, Fantasy, Historical, Magic, Sh...     TV       26   
1                    Comedy, Drama, Romance, Shounen     TV       25   
2  Comedy, Ecchi, Fantasy, Harem, Magic, Romance,...     TV       26   
3                     Adventure, Drama, Supernatural  Movie        1   
4              Comedy, Drama, Mystery, Psychological     TV       24   

   rating_y   members  
0      7.71   33321.0  
1      8.71  133620.0  
2      7.11   76253.0  
3      8.93  466254.0  
4      8.08  156689.0  
RangeIndex(start=0, stop=100000,

In [47]:
df.columns

Index(['user_id', 'anime_id', 'rating_x', 'name', 'genre', 'type', 'episodes',
       'rating_y', 'members'],
      dtype='object')

In [48]:
df.shape

(100000, 9)

In [49]:
mean_rating = df.groupby("anime_id") [["rating_x"]].mean()
mean_rating.sort_values(by="rating_x", ascending=False)

,rating_x
anime_id,
7305,10.0
5763,10.0
15545,10.0
3874,10.0
3868,10.0
...,...
16187,1.0
8654,1.0
3567,1.0


In [50]:
df["rating_x"].describe()

count    100000.000000
mean          7.805400
std           1.576063
min           1.000000
25%           7.000000
50%           8.000000
75%           9.000000
max          10.000000
Name: rating_x, dtype: float64

## Mallinnus

In [51]:
lukija = Reader(rating_scale=(1,10))
data = Dataset.load_from_df(df[["user_id", "name", "rating_x"]], lukija)
data

In [52]:
# Set all data as training set >> tämän voi käyttää, jos riittävästi dataa
from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=0.25)
print(trainset)
print(testset)

[(3009, 'Kara no Kyoukai 1: Fukan Fuukei', 8.0), (12389, 'Kishin Houkou Demonbane', 7.0), (31220, 'Kamisama Hajimemashita', 8.0), (48405, 'Naruto: Shippuuden Movie 1', 7.0), (67696, 'Zettai Junpaku♡Mahou Shoujo', 1.0), (5908, 'Candy Boy Episode: EX01 - Mirai Yohouzu', 6.0), (56656, '.hack//G.U. Trilogy', 8.0), (34756, 'Bakemonogatari', 8.0), (71498, 'Inari, Konkon, Koi Iroha.', 9.0), (69076, 'Chrome Shelled Regios', 7.0), (50804, 'Maoyuu Maou Yuusha', 7.0), (64222, 'Fate/stay night: Unlimited Blade Works 2nd Season', 10.0), (40738, 'Digimon Tamers: Bousou Digimon Tokkyuu', 9.0), (58916, 'Eyeshield 21', 8.0), (49384, 'Sakurasou no Pet na Kanojo', 9.0), (72761, 'Asu no Yoichi!', 4.0), (58990, 'Fate/Zero', 10.0), (37243, 'Big Order', 5.0), (34576, 'Clannad: After Story - Mou Hitotsu no Sekai, Kyou-hen', 9.0), (37719, 'Plastic Memories', 10.0), (26085, 'Elfen Lied', 7.0), (7323, 'Cossette no Shouzou', 7.0), (65212, 'Kämpfer', 7.0), (27851, 'Musaigen no Phantom World', 8.0), (27456, 'Phi Br

In [53]:
# Build and train an algorithm.
# # Tarjotaan trainset algoritmille

# sim_options = {
#     "name": "cosine",
#     "user_based": True,
#     "min_support": True,
#     "min_K": 3
#     }


sim_options = {"user_based": True} # False compute similarities between items, True between customers

algo = KNNBasic(sim_options=sim_options)
algo.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [54]:
preds = algo.test(testset)
preds[0:20]

[Prediction(uid=3009, iid='Kara no Kyoukai 1: Fukan Fuukei', r_ui=8.0, est=7.806186666666667, details={'was_impossible': True, 'reason': 'Not enough neighbors.'}),
 Prediction(uid=12389, iid='Kishin Houkou Demonbane', r_ui=7.0, est=7.806186666666667, details={'was_impossible': True, 'reason': 'Not enough neighbors.'}),
 Prediction(uid=31220, iid='Kamisama Hajimemashita', r_ui=8.0, est=7.806186666666667, details={'was_impossible': True, 'reason': 'Not enough neighbors.'}),
 Prediction(uid=48405, iid='Naruto: Shippuuden Movie 1', r_ui=7.0, est=6.0, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid=67696, iid='Zettai Junpaku♡Mahou Shoujo', r_ui=1.0, est=7.806186666666667, details={'was_impossible': True, 'reason': 'Not enough neighbors.'}),
 Prediction(uid=5908, iid='Candy Boy Episode: EX01 - Mirai Yohouzu', r_ui=6.0, est=7.806186666666667, details={'was_impossible': True, 'reason': 'Not enough neighbors.'}),
 Prediction(uid=56656, iid='.hack//G.U. Trilogy', r_ui=8.0, est

In [55]:
user_id_to_search = 4341  # Replace with the specific user_id
user_data = df[df['user_id'] == user_id_to_search]

print(user_data)


       user_id  anime_id  rating_x       name  \
79130     4341      1412         8  Lupin III   

                                    genre type episodes  rating_y  members  
79130  Action, Adventure, Comedy, Shounen   TV       23      7.76  34373.0  


Mean Absolute Error (MAE)
Root Mean Squared Error (RMSE):
The algorithm shows moderate variability across the three folds, with an average RMSE of approximately 1.6, indicating the typical magnitude of error. MAE provides a complementary perspective, indicating an average absolute error of approximately 1.26.

In [56]:
cross_validate(algo, data, measures=["MAE", "RMSE"], cv=3, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating MAE, RMSE of algorithm KNNBasic on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
MAE (testset)     1.2764  1.2723  1.2753  1.2747  0.0017  
RMSE (testset)    1.6512  1.6394  1.6470  1.6458  0.0049  
Fit time          35.20   33.22   34.25   34.22   0.81    
Test time         2.00    1.94    1.80    1.91    0.08    


{'test_mae': array([1.27636987, 1.27226528, 1.27534041]),
 'test_rmse': array([1.65119128, 1.63935266, 1.64696952]),
 'fit_time': (35.201345443725586, 33.22254538536072, 34.250351428985596),
 'test_time': (1.9986786842346191, 1.9398889541625977, 1.799849271774292)}

In [57]:
from collections import Counter

# Analyze preds for impossible predictions
was_impossible = [pred.details.get('reason', 'Unknown') for pred in preds if pred.details.get('was_impossible')]

# Count the reasons
reason_counts = Counter(was_impossible)

# Output results
print("Total 'was_impossible=True':", len(was_impossible))
print("Reasons and their counts:")
for reason, count in reason_counts.items():
    print(f"{reason}: {count}")



Total 'was_impossible=True': 19237
Reasons and their counts:
Not enough neighbors.: 13918
User and/or item is unknown.: 5319


In [58]:
# Filter out predictions where 'was_impossible' is True
filtered_preds = [pred for pred in preds if not pred.details.get('was_impossible', False)]

# Check the number of filtered predictions
print("Total predictions after filtering:", len(filtered_preds))


Total predictions after filtering: 5763


### SVD yritys

In [59]:
# from surprise import SVD
# import sys
# import surprise

# from recommenders.utils.timer import Timer
# from recommenders.datasets import movielens
# from recommenders.datasets.python_splitters import python_random_split
# from recommenders.evaluation.python_evaluation import (
#     rmse,
#     mae,
#     rsquared,
#     exp_var,
#     map_at_k,
#     ndcg_at_k,
#     precision_at_k,
#     recall_at_k,
#     get_top_k_items,
# )
# from recommenders.models.surprise.surprise_utils import (
#     predict,
#     compute_ranking_predictions,
# )
# from recommenders.utils.notebook_utils import store_metadata


# print(f"System version: {sys.version}")
# print(f"Surprise version: {surprise.__version__}")



In [ ]:
# train, test = python_random_split(df, 0.75)

NameError: name 'python_random_split' is not defined